In [1]:
import pandas as pd
from collections import Counter
import time
# from fruithut.fruithut import *
# from mushroom.mushroom import *
# from foodmart.foodmart import *
from chess.chess import *
# from demo_web.demo_web import *


In [2]:
# sample
# data = {
#     'Tid': ['T1', 'T2', 'T3', 'T4', 'T5', 'T6'],
#     'Items': [['a', 'c', 'd'],
#               ['a', 'b', 'd'],
#               ['b', 'c', 'd', 'e'],
#               ['a', 'd'],
#               ['c', 'd', 'e'],
#               ['a', 'b', 'c', 'd', 'e']]
# }
# df = pd.DataFrame(data)

# data = {
#     'Tid': ['T1', 'T2', 'T3', 'T4', 'T5', 'T6'],
#     'Items': [['apple', 'cherry', 'durian'],
#               ['apple', 'banana', 'durian'],
#               ['banana', 'cherry', 'durian', 'elderberry'],
#               ['apple', 'durian'],
#               ['cherry', 'durian', 'elderberry'],
#               ['apple', 'banana', 'cherry', 'durian', 'elderberry']]
# }

# df = pd.DataFrame(data)
df['Item_Length'] = df['Items'].apply(lambda items: len(items))
len_df = len(df)
df

,Tid,Items,Item_Length
0,T1,"[1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25...",37
1,T2,"[1, 3, 5, 7, 9, 12, 13, 15, 17, 19, 21, 23, 25...",37
2,T3,"[1, 3, 5, 7, 9, 12, 13, 16, 17, 19, 21, 23, 25...",37
3,T4,"[1, 3, 5, 7, 9, 11, 13, 15, 17, 20, 21, 23, 25...",37
4,T5,"[1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25...",37
...,...,...,...
3191,T3192,"[2, 4, 5, 7, 9, 11, 13, 16, 17, 19, 21, 23, 26...",37
3192,T3193,"[2, 4, 5, 7, 9, 11, 13, 16, 17, 19, 21, 23, 26...",37
3193,T3194,"[2, 4, 5, 7, 9, 11, 13, 16, 17, 19, 21, 23, 26...",37
3194,T3195,"[2, 4, 5, 8, 9, 11, 13, 16, 17, 19, 21, 23, 26...",37


In [3]:
unique_items = sorted(df['Items'].explode().unique()) # get unique item => save to list
unique_items

['1',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '2',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '3',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '4',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '5',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '6',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '68',
 '69',
 '7',
 '70',
 '71',
 '72',
 '73',
 '74',
 '75',
 '8',
 '9']

In [4]:
length_transaction = df["Items"].apply(len)
if length_transaction.nunique() == 1:
    hastheSameLengh = True
else:
    hastheSameLengh = False

hastheSameLengh

True

In [5]:
length_transaction = length_transaction.to_list()
length_transaction

[37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,


In [6]:
# calculate stset: {'a': [T1, T2, T4, T6]} - list Tid containing unique item
def cal_stset(df):
    # stset = {} 
    # for item in unique_items:
    #     tid_list = df[df['Items'].apply(lambda items: item in items)]['Tid'].tolist() # create column Items with items in unique item
    #     tid_lengths = [len(df[df['Tid'] == tid]['Items'].iloc[0]) for tid in tid_list if item in df[df['Tid'] == tid]['Items'].iloc[0]]
    #     stset[item] = {"StSet": tid_list, "Length_transaction": tid_lengths} # add value with item_key

    # df_stset = pd.DataFrame.from_dict(stset, orient = 'index').reset_index()
    # df_stset.columns = ["Items", "StSet", "Length_transaction"]
    # df_stset['Items'] = df_stset['Items'].apply(lambda x: [x]) 
    
    df_unpivot = df.explode('Items')
    df_stset = df_unpivot.groupby('Items').agg({'Tid': [list], 'Item_Length': list}).reset_index()
    df_stset.columns = ['Items', 'StSet', 'Length_transaction']
    df_stset['Items'] = df_stset['Items'].apply(lambda x: [x])
    df_stset
    return df_stset

In [7]:
# calculate support - count number of Tid containing unique item
def cal_support(df_stset):
    df_stset['Support'] = df_stset['StSet'].apply(len)
    return df_stset

In [8]:
start_time = time.time()
df_stset = cal_stset(df)
print(df_stset)
end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")

   Items                                              StSet  \
0    [1]  [T1, T2, T3, T4, T5, T6, T7, T8, T9, T10, T11,...   
1   [10]  [T74, T118, T120, T121, T131, T137, T138, T205...   
2   [11]  [T1, T4, T5, T6, T7, T9, T10, T12, T13, T14, T...   
3   [12]  [T2, T3, T8, T11, T15, T16, T18, T22, T27, T36...   
4   [13]  [T1, T2, T3, T4, T5, T6, T7, T8, T9, T10, T11,...   
..   ...                                                ...   
70  [73]  [T33, T34, T35, T36, T37, T38, T39, T40, T46, ...   
71  [74]  [T1, T2, T3, T4, T5, T6, T7, T8, T9, T10, T11,...   
72  [75]  [T560, T561, T562, T563, T564, T565, T566, T56...   
73   [8]  [T130, T132, T215, T218, T320, T407, T410, T50...   
74   [9]  [T1, T2, T3, T4, T5, T6, T7, T8, T9, T10, T11,...   

                                   Length_transaction  
0   [37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 3...  
1   [37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 3...  
2   [37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 3...  
3   [37, 37, 37, 37

In [9]:
df_stset = cal_support(df_stset)
df_stset

,Items,StSet,Length_transaction,Support
0,[1],"[T1, T2, T3, T4, T5, T6, T7, T8, T9, T10, T11,...","[37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 3...",1669
1,[10],"[T74, T118, T120, T121, T131, T137, T138, T205...","[37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 3...",322
2,[11],"[T1, T4, T5, T6, T7, T9, T10, T12, T13, T14, T...","[37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 3...",2129
3,[12],"[T2, T3, T8, T11, T15, T16, T18, T22, T27, T36...","[37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 3...",1067
4,[13],"[T1, T2, T3, T4, T5, T6, T7, T8, T9, T10, T11,...","[37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 3...",1722
...,...,...,...,...
70,[73],"[T33, T34, T35, T36, T37, T38, T39, T40, T46, ...","[37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 3...",851
71,[74],"[T1, T2, T3, T4, T5, T6, T7, T8, T9, T10, T11,...","[37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 3...",2407
72,[75],"[T560, T561, T562, T563, T564, T565, T566, T56...","[37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 3...",789
73,[8],"[T130, T132, T215, T218, T320, T407, T410, T50...","[37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 3...",120


In [10]:
# ex: 'a': {'l(a)': [2, 3, 5], 'n(a)': [1, 2, 1]}
def df_prepare_UBO(df_stset):
    l_item_list = []
    n_item_list = []
    for index, row in df_stset.iterrows():
        item = row['Items']
        length_transaction = row['Length_transaction']

        l_item = sorted(set(length_transaction)) # get unique len(Tid) => sort ascending

        counter = Counter(length_transaction)
        n_item = [counter[i] for i in l_item] # count unique len(Tid) in occupancy_list => same index with l_item
        
        l_item_list.append(l_item)
        n_item_list.append(n_item)
    
    df_stset = df_stset.assign(l_item=l_item_list, n_item=n_item_list)
    return df_stset

In [11]:
# calculate occupancy - O(P) = ∑ T ∈ STSet(P) |P|/|T|
# |P|: len(unique item) itemset {a} =>1
# |T|: len(Tid) 1/3 + 1/3 + 1/2 + 1/5 
def cal_occupancy(df_stset):
    occupancy_data = []
    for index, row in df_stset.iterrows():
        item = row['Items']
        length_transaction = row['Length_transaction']
        total = 0
        for length in length_transaction:
            total += len(item) / length
        occupancy_data.append({'Items': item, 'Occupancy': round(total, 2)})
    
    df_occupancy = pd.DataFrame(occupancy_data)
    df_stset['Occupancy'] = df_occupancy['Occupancy']
    return df_stset

In [12]:
df_stset = cal_occupancy(df_stset)
df_stset

,Items,StSet,Length_transaction,Support,Occupancy
0,[1],"[T1, T2, T3, T4, T5, T6, T7, T8, T9, T10, T11,...","[37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 3...",1669,45.11
1,[10],"[T74, T118, T120, T121, T131, T137, T138, T205...","[37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 3...",322,8.70
2,[11],"[T1, T4, T5, T6, T7, T9, T10, T12, T13, T14, T...","[37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 3...",2129,57.54
3,[12],"[T2, T3, T8, T11, T15, T16, T18, T22, T27, T36...","[37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 3...",1067,28.84
4,[13],"[T1, T2, T3, T4, T5, T6, T7, T8, T9, T10, T11,...","[37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 3...",1722,46.54
...,...,...,...,...,...
70,[73],"[T33, T34, T35, T36, T37, T38, T39, T40, T46, ...","[37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 3...",851,23.00
71,[74],"[T1, T2, T3, T4, T5, T6, T7, T8, T9, T10, T11,...","[37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 3...",2407,65.05
72,[75],"[T560, T561, T562, T563, T564, T565, T566, T56...","[37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 3...",789,21.32
73,[8],"[T130, T132, T215, T218, T320, T407, T410, T50...","[37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 3...",120,3.24


In [13]:
# calculate according to the formula: ni x lx/li
def cal_ubo(l, n):
    total = 0
    for i in range(len(l)):
        total += n[i] * l[0] / l[i]
    return round(total, 2)

In [14]:
# summarize: ∑ni x lx/li => save to list 
def ubo_final(length, number_transaction):
    ubo = []
    for i in range(len(length)): 
        # ex: len = [2,3,5], num_trans = [1,2,1]
        # i = 0 => len = [2,3,5], num_trans = [1,2,1]
        # i = 1 => len = [3,5], num_trans = [2,1]
        # ...
        ubo.append(cal_ubo(length[i:], number_transaction[i:])) # save result cal_ubo for each i => get maxUBO
    return ubo

In [15]:
# get max from summarize => save max value in UBO by key
def calculate_maxUBO(df_UBO):
    df_UBO['List_UBO'] = None # create new column
    df_UBO['Max_UBO'] = None # create new column
    for index, row in df_UBO.iterrows():
        length = row['l_item'] #get list of len(Tid) containing unique item
        number_transaction = row['n_item'] # count unique len(Tid) in occupancy_list
        
        ubo = ubo_final(length, number_transaction) # get list of UBO by i. ex: [2.73, 2.6, 1.0]
        max_ubo = max(ubo) # max list of UBO
        
        df_UBO.at[index, 'List_UBO'] = ubo # save result in df
        df_UBO.at[index, 'Max_UBO'] = max_ubo # save result in df
        
    return df_UBO

In [16]:
# UBO calculation methods: main function
def cal_UBO(df_stset): 
    df_stset = df_prepare_UBO(df_stset)    
    df_stset = calculate_maxUBO(df_stset)
    return df_stset

In [17]:
df_stset = cal_UBO(df_stset)
df_stset

Execution time: 0.5488646030426025 seconds


In [18]:
def mine_hoi_1_itemset(threshold, hastheSameLengh, df_stset):
    C1 = []
    HOI1 = []
    for index, row in df_stset.iterrows():
        item = row['Items'] # 1-itemset in row
        support = row['Support'] # support of 1-itemset
        occupancy = row['Occupancy'] # occuopancy of 1-itemset
        max_ubo = row['Max_UBO'] # max_ubo of 1-itemset
        
        if support >= threshold:
            if hastheSameLengh is False:
                if max_ubo >= threshold:
                    C1.append(item)
                    if occupancy >= threshold:
                        HOI1.append(item)
            else:
                C1.append(item)
                if occupancy >= threshold:
                    HOI1.append(item)
    
    return C1, HOI1

In [19]:
def df_intersection(items1, items2, df_stset):    
    df_intersection = pd.DataFrame(columns=['Items', 'StSet'])
    # set1 = set(items1)
    # set2 = set(items2)
    list_item = sorted(list(set(items1) | set(items2)))
    
    list_occupancy_item = []
    
    for i in list_item:
        list_occupancy_item.append(df_stset[df_stset['Items'].apply(lambda item: i in item)]["StSet"].iloc[0])
        
    intersection_list = set(list_occupancy_item[0])
    for sublist in list_occupancy_item[1:]:
        intersection_list = intersection_list.intersection(sublist)

    intersection_list = sorted(intersection_list, key = lambda x: x[0])
    
    df_intersection = df_intersection.append({'Items': list_item, 'StSet': intersection_list}, ignore_index=True)
    df_intersection['Length_transaction'] = df_intersection['StSet'].apply(lambda x: [len(df[df['Tid'] == tid]['Items'].iloc[0]) for tid in x])
    return df_intersection

In [20]:
# test1 = ['a']
# test2 = ['b']
# df_intersection(test1, test2, df_stset)

In [21]:
def is_same_equivalence_class(P1, P2):
    if len(P1) == len(P2):
        if len(P1) == 1:
            if P1 == P2:
                return False
            else:
                return True
        else:
            if P1 == P2:
                return False
            else:
                new_P1 = P1[:-1]
                new_P2 = P2[:-1]
                if new_P1 == new_P2:
                    return True
                else:
                    return False
    else:
        return False
        # else:
        #     intersection_P1_P2 = sorted(list(set(P1) & set(P2)))
        #     print(intersection_P1_P2)
        #     if len(intersection_P1_P2) == len(P1) - 1:
        #         for element in intersection_P1_P2:
        #             if P1.index(element) != P2.index(element):
        #                 return False
        #         return True
        #     else:
        #         return False

In [22]:
test1 = ['a']
test2 = ['b']
print(is_same_equivalence_class(test1, test2))

True


In [23]:
def cal_occupancy_candidate(items):    
    df_candidate = pd.DataFrame(columns=['Items', 'StSet'])
    
    list_stset_item = []
    
    for i in items:
        list_stset_item.append(df_stset[df_stset['Items'].apply(lambda item: i in item)]["StSet"].iloc[0])
        
    intersection_list = set(list_stset_item[0])
    for sublist in list_stset_item[1:]:
        intersection_list = intersection_list.intersection(sublist)
    
    intersection_list = sorted(intersection_list, key = lambda x: x[0])
    
    df_candidate = df_candidate.append({'Items': items, 'StSet': intersection_list}, ignore_index=True)
    df_candidate['Length_transaction'] = df_candidate['StSet'].apply(lambda x: [len(df[df['Tid'] == tid]['Items'].iloc[0]) for tid in x])
    df_candidate = cal_occupancy(df_candidate)
    return df_candidate
    

In [24]:
# test = ['a','b','d']
# df_test = cal_occupancy_candidate(test)['Occupancy'].iloc[0]
# print(df_test)

In [25]:
def mine_hoi_k_itemset(threshold, hastheSameLengh, CK_minus_1, df_stset):
    CK = []
    HOIK = []
    
    while len(CK_minus_1) > 0:
        P1 = sorted(CK_minus_1[0])
        for P2 in CK_minus_1:
            sorted(P2)
            try:
                if is_same_equivalence_class(P1, P2):
                    P = df_intersection(P1, P2, df_stset)
                    P_items = P['Items'].iloc[0]
                    P_stset = P['StSet'].iloc[0]
                    if len(P_stset) >= threshold:
                        if hastheSameLengh is False:
                            P_ubo = cal_UBO(P)['Max_UBO'].iloc[0]
                            if P_ubo >= threshold:
                                CK.append(P_items)
                        else:
                            CK.append(P_items)
            except:
                continue
                        
        CK_minus_1.pop(0)
    
    for i in CK:
        if cal_occupancy_candidate(i)['Occupancy'].iloc[0] >= threshold:
            HOIK.append(i)
    
    return CK, HOIK
        
                        
    

In [26]:
def mine_depth_hois(threshold, hastheSameLengh, C1, df_stset, HOIS):
    if len(C1) == 0:
        return HOIS
    
    for i in range(len(C1)):
        P1 = C1[i]
        C_l = []
        for j in range(i + 1, len(C1)):
            try:
                P2 = C1[j]
                P = df_intersection(P1, P2, df_stset)
                P_items = P['Items'].iloc[0]
                P_stset = P['StSet'].iloc[0]
                if len(P_stset) >= threshold:
                    if hastheSameLengh is False:
                        P_ubo = cal_UBO(P)['Max_UBO'].iloc[0]
                        if P_ubo >= threshold:
                            C_l.append(P_items)
                    else:
                        C_l.append(P_items)
            except:
                continue

        mine_depth_hois(threshold, hastheSameLengh, C_l, df_stset, HOIS)
        
        for i in C_l:
            if cal_occupancy_candidate(i)['Occupancy'].iloc[0] >= threshold:
                HOIS.append(i)
    
    # return HOIS
        
                        

In [27]:
#test FHOI
HOIS = []
C1 = []
HOI1 = []
CK_minus_1 = []

k = 2 # loop to create 2-itemset
threshold = 0.48
threshold = threshold * len_df # ex: threshold = 25% of len(database)
start_time = time.time()

#create candidate 1 and HOI1 itemset
C1, HOI1 = mine_hoi_1_itemset(threshold, hastheSameLengh, df_stset)

HOIS = HOI1
CK_minus_1 = C1

while CK_minus_1:
    CK, HOIK = mine_hoi_k_itemset(threshold, hastheSameLengh, CK_minus_1, df_stset)
    HOIS.append(HOIK)
    CK_minus_1 = CK
    k += 1

for i in HOIS:
    print(i)

end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")

#update try except


In [ ]:
#test DFHOI
HOIS = []
C1 = []
HOI1 = []
CK_minus_1 = []

threshold = 0.48
threshold = threshold * len_df # ex: threshold = 25% of len(database)
start_time = time.time()

#create candidate 1 and HOI1 itemset
C1, HOI1 = mine_hoi_1_itemset(threshold, hastheSameLengh, df_stset)
HOIS = HOI1
# HOIS = []

HOIS.append(mine_depth_hois(threshold, hastheSameLengh, C1, df_stset, HOIS))

for i in HOIS:
    print(i)

end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")


['item10']
['item13']
['item25']
['item26']
['item10', 'item13']
['item11', 'item26']
['item13', 'item23']
['item13', 'item25']
['item13', 'item26']
['item23', 'item25']
['item25', 'item26']
['item25', 'item28']
None
Execution time: 3.8475990295410156 seconds
